In [ ]:
from datetime import datetime
import pandas as pd
from datetime import timezone

In [ ]:
start_date = datetime.fromisoformat("2018-01-01 00:00").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat("2018-12-31 23:59").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [ ]:
apple_news = pd.read_pickle("pkls/apple_news_data.pkl", compression='infer')
amazon_news = pd.read_pickle("pkls/amazon_news_data.pkl", compression='infer')
apple_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')
amazon_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')

In [128]:
apple_news

,published_time,article_content
16174,1.514832e+09,facebook \nciti analysts say that there is a 4...
15612,1.514855e+09,"quote: in this context avxl is a new apple, im..."
12200,1.514862e+09,2 warren buffett stocks to consider buying now...
15928,1.514862e+09,2 warren buffett stocks to consider buying now...
14085,1.514862e+09,2 warren buffett stocks to consider buying now...
...,...,...
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ..."
61168,1.546322e+09,big trends: gartner iaas mq is down to 6 compa...
41,1.546326e+09,- fundamental analysis - technology is apple s...
60869,1.546332e+09,"new york, oct. 23, 2018 (globe newswire) -- in..."


In [129]:
amazon_news

,published_time,article_content
12886,1.514876e+09,here are some things going on today in your wo...
13571,1.514876e+09,shutterstock photo\nstocks indexes opened the ...
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...
13804,1.514876e+09,what happened shares of many optical networkin...
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...
...,...,...
1348,1.546281e+09,the biggest benefit from living are the lesson...
2553,1.546290e+09,apple stock price\napple (nasdaq: aapl ) was v...
849,1.546306e+09,none of us has a crystal ball that shows us cl...
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ..."


In [131]:
apple_chart["articles"] = [[] for i in range(len(apple_chart))]

In [132]:
amazon_chart

,timestamp,trends
870,1.514876e+09,1
871,1.514963e+09,0
872,1.515049e+09,1
873,1.515136e+09,1
874,1.515395e+09,0
...,...,...
1116,1.545635e+09,0
1117,1.545808e+09,1
1118,1.545894e+09,1
1119,1.545980e+09,0


In [133]:
index=0
row=[]
try:
    for index, row in apple_chart.iterrows():
        time = datetime.fromtimestamp(row['timestamp']).date()
        articles=[]
        for news_index,news_row in apple_news.iterrows():
            if datetime.fromtimestamp(news_row['published_time']).date() > time:
                break
            if datetime.fromtimestamp(news_row['published_time']).date() == time:
                articles.append(news_row['article_content'])
        apple_chart.at[index, 'articles'] = articles
except Exception as e:
    print(e)
    print(row,index)

In [134]:
apple_chart

,timestamp,trends,articles
870,1.514876e+09,1,"[by ciara linnane, marketwatch\nanalysts said ..."
871,1.514963e+09,0,[netflix ( nflx ) shares rocketed for the seco...
872,1.515049e+09,1,[by nigam arora\nnigam arora says computer pro...
873,1.515136e+09,1,[shutterstock photo top tech stocks\nmsft +1.1...
874,1.515395e+09,0,[alphabet ( nasdaq:goog ) ( nasdaq:googl ) sub...
...,...,...,...
1116,1.545635e+09,0,"[by richard suttmeier updated dec 24, 2018\nap..."
1117,1.545808e+09,1,"[apple inc. (aapl) closed at $157.17, up $10.3..."
1118,1.545894e+09,1,[apple ( nasdaq:aapl ) surprised investors bac...
1119,1.545980e+09,0,"[by david goldman, cnn business\ncapping a wee..."


In [143]:
len(apple_chart['articles'][873])

123